<a href="https://colab.research.google.com/github/sowrabh-adiga/all-Machine-learning-codes/blob/main/Hyperparameter_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

in documentation the hyperparameters which are to be tuned first are mentioned, have to look at them first before tuning other parameters

Accordingly we are going adjust:

*   `max_depth`
*   `max_features`
* `min_samples_leaf`
* `min_samples_split`
* `n_estimators`




In [ ]:
def evaluate_preds(y_true, y_preds):
  """
  Performs evaluation comparision on y_true labels vs y_pred labels
  """
  accuracy = accuracy_score(y_true,y_preds)
  precision = precision_score(y_true,y_preds)
  recall = recall_score(y_true,y_preds)
  f1 = f1_score(y_true,y_preds)
  metric_dict = {'accuracy':round(accuracy,2),
                 'precision':round(precision,2),
                 'recall':round(recall,2),
                 'f1':round(f1,2)}
  print(f'ACC: {accuracy * 100:.2f}%')
  print(f'Precision: {precision * 100:.2f}%')
  print(f'Recall: {recall * 100:.2f}%')
  print(f'F1: {f1 * 100:.2f}%')

  return metric_dict


In [ ]:
def train_test_val_split(df,x,y):
  train_split = round(0.7 * len(df)) #70% for training
  valid_split = round(train_split + 0.15 * len(df)) #15% for validation, rest for test
  x_train,y_train = x[:train_split],y[:train_split]
  x_valid,y_valid = x[train_split:valid_split],y[train_split:valid_split]
  x_test,y_test = x[valid_split:],y[valid_split:]
  return x_train,y_train,x_valid,y_valid,x_test,y_test


In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
import random

np.random.seed(42)

#shuffle data
heart_disease = pd.read_csv('heart-disease.csv')
heart_disease_shuffled = heart_disease.sample(frac=1)  

#split X & Y

x = heart_disease_shuffled.drop("target",axis=1)
y = heart_disease_shuffled['target']

#Split the data into train, validation & test sets
x_train,y_train,x_valid,y_valid,x_test,y_test = train_test_val_split(heart_disease_shuffled,x,y)

#model
clf = RandomForestClassifier()
clf.fit(x_train,y_train)

#Predict
y_preds = clf.predict(x_valid)

#Evaluate
baseline_metrics = evaluate_preds(y_valid,y_preds)
baseline_metrics


FileNotFoundError: ignored

In [ ]:
clf.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}


##***`Decision Tree:`*** 

resource : https://youtu.be/7VeUPuFGJHk
1.   Internal Nodes / Nodes : arrows  enter and leave nodes
2.   Leaf Nodes : Arrows enter the node, present at the end of tree
3.   Intuitive
4.   Impurity: measured by Gini impurity
5.   Trees are built on Gini impurity.


##***`Random Forest:`***

resource : https://youtu.be/J4Wdy0Wc_xQ


1.   Create a bootstrapped dataset:create a dataset by taking random sample from original data set, repetations are allowed, the length of dataset has to be same.
2.   Run decision tree on it, bootstrap and run decision tree again, repeat
3.   The predicting data is taken and run through all the trees created and aggregate of the results is used to get the results (like votes).
4.   If a sample is missed during bootstrap, it's "out-of-bag sample", is used for validation, if validation set is not created manually.







In [ ]:
clf_3 = RandomForestClassifier(max_depth=10,)
clf_3.fit(x_train,y_train)

#Predict
y_preds_3 = clf_3.predict(x_valid)

#Evaluate
baseline_metrics = evaluate_preds(y_valid,y_preds_3)
baseline_metrics

ACC: 82.22%
Precision: 81.48%
Recall: 88.00%
F1: 84.62%


{'accuracy': 0.82, 'f1': 0.85, 'precision': 0.81, 'recall': 0.88}

# Automated tuning of model  by scikit learn methods 

##Randomized search CV (CrossValidation): 
###randomly select n_iter number of values and find best possible combination.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
grid = {"n_estimators":[100,5000,1000,1200],
        "max_depth": [None,5,10,20,30],
        "max_features": ["auto","sqrt"],
        "min_samples_split":[2,4,6],
        "min_samples_leaf":[1,2,4]}

np.random.seed(42)

#get x and y
x = heart_disease_shuffled.drop("target",axis=1)
y = heart_disease_shuffled['target']


#split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)

#instantiate
clf = RandomForestClassifier(n_jobs=1)

#Setup Randomized search cv
rs_clf = RandomizedSearchCV(estimator=clf,
                            param_distributions=grid,
                            n_iter=10, #no. of iterations
                            cv =5,
                            verbose=2)

#fit the Randomize seach vs of clf
rs_clf.fit(x_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] n_estimators=1000, min_samples_split=6, min_samples_leaf=2, max_features=sqrt, max_depth=None 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  n_estimators=1000, min_samples_split=6, min_samples_leaf=2, max_features=sqrt, max_depth=None, total=   1.7s
[CV] n_estimators=1000, min_samples_split=6, min_samples_leaf=2, max_features=sqrt, max_depth=None 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.7s remaining:    0.0s


[CV]  n_estimators=1000, min_samples_split=6, min_samples_leaf=2, max_features=sqrt, max_depth=None, total=   1.7s
[CV] n_estimators=1000, min_samples_split=6, min_samples_leaf=2, max_features=sqrt, max_depth=None 
[CV]  n_estimators=1000, min_samples_split=6, min_samples_leaf=2, max_features=sqrt, max_depth=None, total=   1.7s
[CV] n_estimators=1000, min_samples_split=6, min_samples_leaf=2, max_features=sqrt, max_depth=None 
[CV]  n_estimators=1000, min_samples_split=6, min_samples_leaf=2, max_features=sqrt, max_depth=None, total=   1.7s
[CV] n_estimators=1000, min_samples_split=6, min_samples_leaf=2, max_features=sqrt, max_depth=None 
[CV]  n_estimators=1000, min_samples_split=6, min_samples_leaf=2, max_features=sqrt, max_depth=None, total=   1.7s
[CV] n_estimators=1000, min_samples_split=2, min_samples_leaf=2, max_features=sqrt, max_depth=20 
[CV]  n_estimators=1000, min_samples_split=2, min_samples_leaf=2, max_features=sqrt, max_depth=20, total=   1.7s
[CV] n_estimators=1000, min_s

KeyboardInterrupt: ignored

In [ ]:
rs_clf.best_params_

{'max_depth': 5,
 'max_features': 'sqrt',
 'min_samples_leaf': 4,
 'min_samples_split': 2,
 'n_estimators': 1200}

In [ ]:
#make predictions
rs_y_preds = rs_clf.predict(x_test)

#Evaluate
rs_metrics = evaluate_preds(y_test,rs_y_preds)

ACC: 83.61%
Precision: 78.12%
Recall: 89.29%
F1: 83.33%


In [ ]:
!lscpu

Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              2
On-line CPU(s) list: 0,1
Thread(s) per core:  2
Core(s) per socket:  1
Socket(s):           1
NUMA node(s):        1
Vendor ID:           GenuineIntel
CPU family:          6
Model:               63
Model name:          Intel(R) Xeon(R) CPU @ 2.30GHz
Stepping:            0
CPU MHz:             2299.998
BogoMIPS:            4599.99
Hypervisor vendor:   KVM
Virtualization type: full
L1d cache:           32K
L1i cache:           32K
L2 cache:            256K
L3 cache:            46080K
NUMA node0 CPU(s):   0,1
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm invpcid_single ssbd ibrs 

##GridsearchCV: 
###Brute force method

In [ ]:
grid = {"n_estimators":[100,5000,1000,1200],
        "max_depth": [None,5,10,20,30],
        "max_features": ["auto","sqrt"],
        "min_samples_split":[2,4,6],
        "min_samples_leaf":[1,2,4]}
grid

{'max_depth': [None, 5, 10, 20, 30],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 4, 6],
 'n_estimators': [100, 5000, 1000, 1200]}

In [ ]:
grid_2 = {'max_depth': [None, 5, 10],
          'max_features': ['auto', 'sqrt'],
          'min_samples_leaf': [2, 3, 4, 5],
          'min_samples_split': [ 3, 4, 6],
          'n_estimators': [ 500, 1000, 1200, 1500]}

In [ ]:
from sklearn.model_selection import GridSearchCV, train_test_split



np.random.seed(42)

#get x and y
x = heart_disease_shuffled.drop("target",axis=1)
y = heart_disease_shuffled['target']


#split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)

#instantiate
clf = RandomForestClassifier(n_jobs=1)

#Setup GridSeachCv
gs_clf = GridSearchCV(estimator=clf,
                      param_grid=grid_2,
                      cv =5,
                      verbose=2)

#fit the Randomize seach vs of clf
gs_clf.fit(x_train,y_train)

In [ ]:
rs_clf.best_params_

{'max_depth': None,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 4,
 'n_estimators': 100}

In [ ]:
#make predictions
gs_y_preds = rs_clf.predict(x_test)

#Evaluate
gs_metrics = evaluate_preds(y_test,rs_y_preds)

ACC: 80.33%
Precision: 75.00%
Recall: 85.71%
F1: 80.00%


compare_metrics : take the dictionary and plot the attributes

#Saving and loading ML models

###2 ways :


*   Python Pickle (ok to use for smaller models)
*   Job lib module (large models with large arrays use this) [recommended]




 ### Pickling: saving

In [ ]:
import pickle

pickle.dump(rs_clf, open("rs_random_forest_model.pkl","wb"))

 ### Pickling: Load 

In [ ]:
pickled_model = pickle.load(open("rs_random_forest_model.pkl","rb"))
# ------------------- this model is a regular model, all method working on the model before will also work now --------------------#

### Job lib :saving

In [ ]:
from joblib import dump, load

dump(rs_clf, filename="rs_random_forest.joblib")

### Job lib : Load

In [ ]:
loaded_job_model = load(filename="rs_random_forest.joblib")


# Putting it all togather 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

data = pd.read_csv('car-sales-extended-missing-data.csv')
data

,Make,Colour,Odometer (KM),Doors,Price
0,Honda,White,35431.0,4.0,15323.0
1,BMW,Blue,192714.0,5.0,19943.0
2,Honda,White,84714.0,4.0,28343.0
3,Toyota,White,154365.0,4.0,13434.0
4,Nissan,Blue,181577.0,3.0,14043.0
...,...,...,...,...,...
995,Toyota,Black,35820.0,4.0,32042.0
996,NaN,White,155144.0,3.0,5716.0
997,Nissan,Blue,66604.0,4.0,31570.0
998,Honda,White,215883.0,4.0,4001.0


In [ ]:
data.dtypes

Make              object
Colour            object
Odometer (KM)    float64
Doors            float64
Price            float64
dtype: object

In [ ]:
data.isna().sum()

Make             49
Colour           50
Odometer (KM)    50
Doors            50
Price            50
dtype: int64

steps we need to do:
1. Fill missing data2
2. Enumerate categorical data
3. Build model

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

#Modelling
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split,GridSearchCV

#setup Random seed
np.random.seed(42)

# Import data without NULL
data.dropna(subset=['Price'], inplace=True)

#Define features in categorical pipeline
categorical_features = ['Make','Colour']
categorical_transformer = Pipeline(steps=[
         ("imputer", SimpleImputer(strategy = "constant", fill_value='missing')),
          ('onehot', OneHotEncoder(handle_unknown="ignore"))
])

door_feature = ['Doors']
door_transformer = Pipeline(steps=[
         ("imputer", SimpleImputer(strategy = "constant", fill_value= 4 ))                                  
])

numeric_features = ['Odometer (KM)']
numeric_transformer = Pipeline(steps=[
          ("imputer", SimpleImputer(strategy = 'mean'))
])


#Do preprocessing value 
preprocessor = ColumnTransformer(transformers = [
                                                ('cat', categorical_transformer, categorical_features),
                                                ('door', door_transformer, door_feature),
                                                ('num', numeric_transformer,numeric_features)
])

#preprocessing and modelling pipline
model = Pipeline(steps =[('preprocessor',preprocessor),
                         ('model', RandomForestRegressor())])

#split data
x = data.drop('Price', axis=1)
y = data["Price"]


x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)

#Fit and score model
model.fit(x_train,y_train)
model.score(x_test,y_test)

0.22188417408787875

using `RandomizedSearchCV` or `GridSearchCV` with `Pipeline`

In [ ]:
from sklearn.model_selection import GridSearchCV

pipe_grid = {
    'preprocessor__num__imputer__strategy': ['mean','median'],
    'model__n_estimators': [100,1000],
    'model__max_depth':[None,5],
    'model__max_features':['auto'],
    'model__min_samples_split': [2, 4]
}

gs_model = GridSearchCV(model,pipe_grid,cv=5,verbose=2)
gs_model.fit(x_train,y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV] model__max_depth=None, model__max_features=auto, model__min_samples_split=2, model__n_estimators=100, preprocessor__num__imputer__strategy=mean 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  model__max_depth=None, model__max_features=auto, model__min_samples_split=2, model__n_estimators=100, preprocessor__num__imputer__strategy=mean, total=   0.3s
[CV] model__max_depth=None, model__max_features=auto, model__min_samples_split=2, model__n_estimators=100, preprocessor__num__imputer__strategy=mean 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s


[CV]  model__max_depth=None, model__max_features=auto, model__min_samples_split=2, model__n_estimators=100, preprocessor__num__imputer__strategy=mean, total=   0.3s
[CV] model__max_depth=None, model__max_features=auto, model__min_samples_split=2, model__n_estimators=100, preprocessor__num__imputer__strategy=mean 
[CV]  model__max_depth=None, model__max_features=auto, model__min_samples_split=2, model__n_estimators=100, preprocessor__num__imputer__strategy=mean, total=   0.3s
[CV] model__max_depth=None, model__max_features=auto, model__min_samples_split=2, model__n_estimators=100, preprocessor__num__imputer__strategy=mean 
[CV]  model__max_depth=None, model__max_features=auto, model__min_samples_split=2, model__n_estimators=100, preprocessor__num__imputer__strategy=mean, total=   0.3s
[CV] model__max_depth=None, model__max_features=auto, model__min_samples_split=2, model__n_estimators=100, preprocessor__num__imputer__strategy=mean 
[CV]  model__max_depth=None, model__max_features=auto, 

[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:  1.5min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('preprocessor',
                                        ColumnTransformer(n_jobs=None,
                                                          remainder='drop',
                                                          sparse_threshold=0.3,
                                                          transformer_weights=None,
                                                          transformers=[('cat',
                                                                         Pipeline(memory=None,
                                                                                  steps=[('imputer',
                                                                                          SimpleImputer(add_indicator=False,
                                                                                                        copy=True,
                                    

In [ ]:
gs_model.score(x_test,y_test)

0.3339554263158365